<a href="https://colab.research.google.com/github/garmartirosy/GithubActions/blob/main/ImportCommodityFactor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymrio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 17.8 MB/s eta 0:00:00
  Created wheel for pymrio: filename=pymrio-0.5.4-py3-none-any.whl size=140078 sha256=2a85d2a10c4886868db87907a5cfa6c620810b4ed795f33bad7ae71cc12a47b6
  Stored in directory: /root/.cache/pip/wheels/f7/a2/98/de741b07dff4ebf8f79d17cd943f9d64f77c4fb2f1e08b7816
Successfully built pymrio
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.1.5
    Uninstalling openpyxl-3.1.5:
      Successfully uninstalled openpyxl-3.1.5


In [ ]:
import pandas as pd
import pymrio

pymrio.download_exiobase3(storage_folder='content', system='pxp', years=[2012])


exio_data = pymrio.parse_exiobase3(path="/content/content/IOT_2012_pxp.zip")


# Step 1: Reset the index
d_imp_reg_df = exio_data.satellite.D_imp_reg.reset_index()

# Step 2: Melt the DataFrame to long format
import_long_df = d_imp_reg_df.melt(
    id_vars=['stressor'],  # Keep 'stressor' fixed
    var_name='Region',     # Region is derived from columns
    value_name='ImportAmount'  # Values correspond to import amounts
)

# Step 3: Filter rows with non-zero import amounts
import_long_df = import_long_df[import_long_df['ImportAmount'] != 0]

# Step 4: Add FlowUUID
import_long_df['FlowUUID'] = import_long_df.apply(
    lambda row: f"{row['Region']}_{row['stressor']}", axis=1
)


country_df = pd.read_csv('exio_country_names.csv')  # Ensure the file is in the correct directory

# Step 10: Merge the country names with the commodity_factor_df
final_df = import_long_df.merge(
    country_df,
    left_on='Region',        # Match 'Region' from commodity_factor_df
    right_on='CountryCode',  # Match 'CountryCode' from country_df
    how='left'               # Perform a left join to keep all rows from commodity_factor_df
)

# Step 11: Drop the 'CountryCode' column after merging (optional)
final_df.drop(columns=['CountryCode'], inplace=True)

# Step 12: Display the final merged DataFrame

# Optional: Save the final DataFrame to a new CSV file
final_df

,stressor,Region,ImportAmount,FlowUUID,Country
0,Taxes less subsidies on products purchased: Total,AT,6.378177e+03,AT_Taxes less subsidies on products purchased:...,Austria
1,Other net taxes on production,AT,2.004912e+03,AT_Other net taxes on production,Austria
2,"Compensation of employees; wages, salaries, & ...",AT,2.978250e+03,"AT_Compensation of employees; wages, salaries,...",Austria
3,"Compensation of employees; wages, salaries, & ...",AT,2.284562e+04,"AT_Compensation of employees; wages, salaries,...",Austria
4,"Compensation of employees; wages, salaries, & ...",AT,1.726219e+04,"AT_Compensation of employees; wages, salaries,...",Austria
...,...,...,...,...,...
50282,Energy Carrier Net TMAR,WM,1.052491e+05,WM_Energy Carrier Net TMAR,"Rest of World, Middle East"
50283,Energy Carrier Net TOTH,WM,4.704499e+04,WM_Energy Carrier Net TOTH,"Rest of World, Middle East"
50284,Energy Carrier Net TRAI,WM,2.426882e+04,WM_Energy Carrier Net TRAI,"Rest of World, Middle East"
50285,Energy Carrier Net TROA,WM,7.254860e+05,WM_Energy Carrier Net TROA,"Rest of World, Middle East"
